# 卷积神经网络手写数字识别
使用卷积神经网络来完成手写数字识别，卷积神经网络主要分为了以下几层：
+ 卷积层
+ 池化层
  + 最大池化
  + 平均池化
+ 全连接层

## 卷积层
卷积层是进行卷积运算，卷积运算有
+ `kernel_size`: 卷积核的大小
+ `stride`: 步长，即每次的偏移量
+ `padding`: 在图像的两端周围补0的个数

## 池化层
池化层(Pooling Layer)又叫汇聚层，或者叫子采样层(Subsampling Layer),其作用是进行特征选择，降低特征的数量，从而减少参数的数量。将前面输出的特征映射划分为多个区域，然后对这些区域运用池化(汇聚)操作，就可以显著的降低特征的维度。
+ 最大池化(汇聚)：对于某个区域$R^d_{m,n}$,选择这个区域的所有神经元的最大的活性值作为这个区域的表示
+ 平均池化(汇聚)：一般取区域内所有神经元值的平均值

## 全连接层
即将最后池化的结果展开，神经元之间是全连接的。



In [33]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# 加载数据集

In [34]:
train_dataset = torchvision.datasets.MNIST(root='../data',download=True,train=True,transform=transforms.ToTensor())
test_dataset = torchvision.datasets.MNIST(root='../data',download=True,train=False,transform=transforms.ToTensor())
print(train_dataset.data.shape)
print(train_dataset.classes)

torch.Size([60000, 28, 28])
['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']


# 设置超参数以及DataLoader

In [35]:
nums_class = 10
num_epoch = 5
learning_rate = 0.001
batch_size = 100

train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

# 建立模型
两层卷积，再加上全连接

In [36]:
class CNN(nn.Module):
    def __init__(self, nums_class) -> None:
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            # 原始图片是 28*28的，p=2,周围补两位的0，就变成了32*32
            # 进行卷积运算，用16个5*5的卷积核进行卷积（符号记为: 16@5*5）
            # 卷积后是 32 - 5 + 1 = 28，成为了16@28*28
            nn.Conv2d(1,16,kernel_size=5,padding=2,stride=1),
            nn.BatchNorm2d(16), # 卷积结束后进行归一化，防止过拟合
            nn.ReLU(),
            # 利用最大池化操作，池化核是2，步长是2
            # 也就是将整个输出的大小除二
            # 结束后输出的是 16@14*14
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.layer2 = nn.Sequential(
            # 上一层卷积输出的是16@14*14
            # 一般的卷积都是减少输出的大小，增加生成的个数
            # 所以，在此处依然进行一样的卷积和池化，使得变成32@7*7
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5, stride=1,padding=2), # 32@14*14
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 32@7*&
        )
        # 全连接层
        # 上面是 32@7*7,使用linear与十个分类全连接，展开操作在训练的时候reshape
        self.fc = nn.Linear(32*7*7, nums_class)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0),-1)
        out = self.fc(out)
        return out

# 创建模型，开始计算

In [37]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
cnn = CNN(nums_class).to(device)

# 定义损失函数
loss_fn = nn.CrossEntropyLoss()
# 定义优化器
optmizer = torch.optim.Adam(cnn.parameters(),lr=learning_rate)

# 开始训练
total_step = len(train_dataloader)
for epoch in range(num_epoch):
    for i,(images,labels) in enumerate(train_dataloader):
        images = images.to(device)
        labels = labels.to(device)

        # 前向传播
        outputs = cnn(images)
        # 计算损失值
        loss = loss_fn(outputs, labels)

        # 反向传播
        optmizer.zero_grad()
        loss.backward()
        optmizer.step()
        
        if (i + 1) % 100 == 0:
            print("Epoch [{}/{}],Step [{}/{}], loss:{:.4f}".format(epoch+1, num_epoch, i+1,total_step,loss.item() ))

Epoch [0/5],Step [100/600], loss:0.1600
Epoch [0/5],Step [200/600], loss:0.1311
Epoch [0/5],Step [300/600], loss:0.1025
Epoch [0/5],Step [400/600], loss:0.0560
Epoch [0/5],Step [500/600], loss:0.0805
Epoch [0/5],Step [600/600], loss:0.0469
Epoch [1/5],Step [100/600], loss:0.0898
Epoch [1/5],Step [200/600], loss:0.1088
Epoch [1/5],Step [300/600], loss:0.0506
Epoch [1/5],Step [400/600], loss:0.0332
Epoch [1/5],Step [500/600], loss:0.0113
Epoch [1/5],Step [600/600], loss:0.0452
Epoch [2/5],Step [100/600], loss:0.0259
Epoch [2/5],Step [200/600], loss:0.0497
Epoch [2/5],Step [300/600], loss:0.0866
Epoch [2/5],Step [400/600], loss:0.0060
Epoch [2/5],Step [500/600], loss:0.0189
Epoch [2/5],Step [600/600], loss:0.0050
Epoch [3/5],Step [100/600], loss:0.0401
Epoch [3/5],Step [200/600], loss:0.0076
Epoch [3/5],Step [300/600], loss:0.0520
Epoch [3/5],Step [400/600], loss:0.0071
Epoch [3/5],Step [500/600], loss:0.0199
Epoch [3/5],Step [600/600], loss:0.0745
Epoch [4/5],Step [100/600], loss:0.0258


# 测试准确率

In [38]:
cnn.eval()  # 与no_grad配合运算，关闭梯度计算和归一化的不同
total_num = 0
correct_num = 0
with torch.no_grad():
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = cnn(images)

        _,predicted = torch.max(outputs, 1)
        total_num += outputs.size(0)
        correct_num += (predicted == labels).sum()
    print("准确率为:{:.3f}%".format(100*correct_num/total_num))

准确率为:98.820%
